<a href="https://colab.research.google.com/github/AI4Finance-LLC/ElegantRL/blob/master/eRL_demo_StockTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Application in ElegantRL**






# **Part 1: Problem Formulation**
Formally, we model stock trading as a Markov Decision Process (MDP), and formulate the trading objective as maximization of expected return:



*   **State s = [b, p, h]**: a vector that includes the remaining balance b, stock prices p, and stock shares h. p and h are vectors with D dimension, where D denotes the number of stocks. 
*   **Action a**: a vector of actions over D stocks. The allowed actions on each stock include selling, buying, or holding, which result in decreasing, increasing, or no change of the stock shares in h, respectively.
*   **Reward r(s, a, s’)**: The asset value change of taking action a at state s and arriving at new state s’.
*   **Policy π(s)**: The trading strategy at state s, which is a probability distribution of actions at state s.
*   **Q-function Q(s, a)**: the expected return (reward) of taking action a at state s following policy π.
*   **State-transition**: After taking the actions a, the number of shares h is modified, as shown in Fig 3, and the new portfolio is the summation of the balance and the total value of the stocks.

# **Part 2: Stock Trading Environment Design**

**State Space and Action Space**


*   **State Space**: We use a 181-dimensional vector consists of seven parts of information to represent the state space of multiple stocks trading environment: [b, p, h, M, R, C, X], where b is the balance, p is the stock prices, h is the number of shares, M is the Moving Average Convergence Divergence (MACD), R is the Relative Strength Index (RSI), C is the Commodity Channel Index (CCI), and X is the Average Directional Index (ADX).
*   **Action Space**: As a recap, we have three types of actions: selling, buying, and holding for a single stock. We use the negative value for selling, positive value for buying, and zero for holding. In this case, the action space is defined as {-k, …, -1, 0, 1, …, k}, where k is the maximum share to buy or sell in each transaction.


**Easy-to-customize Features**


*   **initial_capital**: the initial capital that the user wants to invest.
*   **tickers**: the stocks that the user wants to trade with.
*   **initial_stocks**: the initial amount of each stock and the default could be zero.
*   **buy_cost_pct, sell_cost_pct**: the transaction fee of each buying or selling transaction.
*   **max_stock**: the user is able to define the maximum number of stocks that are allowed to trade per transaction. Users can also set the maximum percentage of capitals to invest in each stock.
*   **tech_indicator_list**: the list of financial indicators that are taken into account, which is used to define a state.
*   **start_date, start_eval_date, end_eval_date**: the training and backtesting time intervals. Thee time dates (or timestamps) are used, once the training period is specified, the rest is backtesting.


# **Part 3: Install ElegantRL and related packages**

In [1]:
!pip install git+https://github.com/AI4Finance-LLC/ElegantRL.git
!pip install yfinance stockstats

  Cloning https://github.com/AI4Finance-LLC/ElegantRL.git to /tmp/pip-req-build-n9h1a_uw
  Running command git clone --filter=blob:none -q https://github.com/AI4Finance-LLC/ElegantRL.git /tmp/pip-req-build-n9h1a_uw
  Resolved https://github.com/AI4Finance-LLC/ElegantRL.git to commit ede6a0fbc4059d5117f4c9697028c82af5213bd9
  Preparing metadata (setup.py) ... done
  Using cached gym-0.21.0-py3-none-any.whl
     |████████████████████████████████| 10.3 MB 26.0 MB/s            
  Using cached pybullet-3.2.0-cp37-cp37m-manylinux1_x86_64.whl (89.3 MB)
  Using cached torch-1.10.0-cp37-cp37m-manylinux1_x86_64.whl (881.9 MB)
     |████████████████████████████████| 60.3 MB 105.5 MB/s            
  Using cached box2d_py-2.3.8-cp37-cp37m-manylinux1_x86_64.whl (448 kB)
  Using cached cloudpickle-2.0.0-py3-none-any.whl (25 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     |████████████████████████████████| 3.1 MB 45.0 MB/s            
     |████████████████████████████████| 1.1 MB 84.7

In [2]:
import sys
import os
print(sys.version)
!pip freeze


print(sys.path)

3.7.5 (default, Feb 23 2021, 13:22:40) 
[GCC 8.4.0]
argcomplete==1.12.3
argon2-cffi==21.1.0
attrs==21.2.0
backcall==0.2.0
bleach==4.1.0
box2d-py==2.3.8
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.7
cloudpickle==2.0.0
cycler==0.11.0
debugpy==1.5.1
decorator==5.1.0
defusedxml==0.7.1
elegantrl @ git+https://github.com/AI4Finance-LLC/ElegantRL.git@ede6a0fbc4059d5117f4c9697028c82af5213bd9
entrypoints==0.3
gym==0.21.0
idna==3.3
importlib-metadata==4.8.1
importlib-resources==5.4.0
int-date==0.1.8
ipykernel==6.5.0
ipython==7.29.0
ipython-genutils==0.2.0
ipywidgets==7.6.5
jedi==0.18.0
Jinja2==3.0.2
jsonschema==4.2.0
jupyter==1.0.0
jupyter-client==7.0.6
jupyter-console==6.4.0
jupyter-core==4.9.1
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.2
kiwisolver==1.3.2
lxml==4.6.4
MarkupSafe==2.0.1
matplotlib==3.4.3
matplotlib-inline==0.1.3
mistune==0.8.4
multitasking==0.0.9
nbclient==0.5.4
nbconvert==6.2.0
nbformat==5.1.3
nest-asyncio==1.5.1
notebook==6.4.5
numpy==1.21.4
opencv-python==

In [ ]:
import site
print(site.getsitepackages())

# **Part 4: Import Packages**


*   **ElegantRL**
*   **yfinance**: yfinance aims to solve this problem by offering a reliable, threaded, and Pythonic way to download historical market data from Yahoo! finance.
*   **StockDataFrame**: stockstats inherits and extends pandas.DataFrame to support Stock Statistics and Stock Indicators.



In [3]:
from elegantrl.run import *
from elegantrl.agent import AgentPPO, AgentDDPG
from elegantrl.envs.FinRL.StockTrading import StockTradingEnv, check_stock_trading_env
import yfinance as yf
from stockstats import StockDataFrame as Sdf

# **Part 5: Specify Agent and Environment**

*   **args.agent**: firstly chooses one DRL algorithm to use from agent.py. In this application, we prefer to choose DDPG and PPO agent.
*   **args.env**: creates the environment, and the user can either customize own environment or preprocess environments from OpenAI Gym and PyBullet Gym from env.py. In this application, we create the self-designed stock trading environment.


> Before finishing initialization of **args**, please see Arguments() in run.py for more details about adjustable hyper-parameters.




In [13]:
# Environment
"""
tickers = [
  'AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ALGN', 'ALXN', 'AMAT', 'AMD', 'AMGN',
  'AMZN', 'ASML', 'ATVI', 'BIIB', 'BKNG', 'BMRN', 'CDNS', 'CERN', 'CHKP', 'CMCSA',
  'COST', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'DLTR', 'EA', 'EBAY', 'FAST',
  'FISV', 'GILD', 'HAS', 'HSIC', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG',
  'JBHT', 'KLAC', 'LRCX', 'MAR', 'MCHP', 'MDLZ', 'MNST', 'MSFT', 'MU', 'MXIM',
  'NLOK', 'NTAP', 'NTES', 'NVDA', 'ORLY', 'PAYX', 'PCAR', 'PEP', 'QCOM', 'REGN',
  'ROST', 'SBUX', 'SIRI', 'SNPS', 'SWKS', 'TTWO', 'TXN', 'VRSN', 'VRTX', 'WBA',
  'WDC', 'WLTW', 'XEL', 'XLNX']  # finrl.config.NAS_74_TICKER
"""

tickers = ['COKE', 'AAPL']

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1e2
initial_capital = 1e6
initial_stocks = np.zeros(len(tickers), dtype=np.float32)
buy_cost_pct = 1e-3
sell_cost_pct = 1e-3
start_date = '2008-03-19'
start_eval_date = '2016-01-01'
end_eval_date = '2021-01-01'

# Agent
agent = AgentPPO() # AgentSAC(), AgentTD3(), AgentDDPG()
agent.if_use_gae = True
agent.lambda_entropy = 0.04
agent.if_on_policy = True

# Environment
env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=False)
env_eval = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True)

env.target_reward = 3
env_eval.target_reward = 3

args = Arguments(env, agent)




# Hyperparameters
args.gamma = gamma
args.break_step = int(2e5)
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = args.max_step * 4
args.batch_size = 2 ** 10
args.repeat_times = 2 ** 3
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = False
args.rollout_num = 2 # the number of rollout workers (larger is not always faster)

# **Part 6: Train and Evaluate the Agent**

> The training and evaluating processes are all finished inside function **train_and_evaluate_mp()**, and the only parameter for it is **args**. It includes the fundamental objects in DRL:

*   agent,
*   environment.

> And it also includes the parameters for training-control:

*   batch_size,
*   target_step,
*   reward_scale,
*   gamma, etc.

> The parameters for evaluation-control:

*   break_step,
*   random_seed, etc.






In [14]:
train_and_evaluate_mp(args) # the training process will terminate once it reaches the target reward.

| Remove cwd: ./AgentPPO_StockTradingEnv-v1_(0,)
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  1.57e+04    2.04 |
0  1.57e+04    2.04 |    2.04    0.0   1960     0 |    0.02   0.58   0.04  -0.50
0  1.14e+05    2.08 |
0  1.14e+05    2.08 |    2.08    0.0   1960     0 |    0.05   0.57   0.09  -0.50
0  1.22e+05    2.09 |
0  1.22e+05    2.09 |    2.09    0.0   1960     0 |    0.06   0.53  -0.01  -0.50
0  1.29e+05    2.11 |
0  1.29e+05    2.11 |    2.11    0.0   1960     0 |    0.09   0.52  -0.06  -0.50
0  1.37e+05    2.15 |
0  1.37e+05    2.15 |    2.15    0.0   1960     0 |    0.03   0.63   0.02  -0.50
0  1.45e+05    2.15 |    2.14    0.0   1960     0 |    0.07   0.53   0.02  -0.50
0  1.53e+05    2.20 |
0  1.53e+05    2.20 |    2.20    0.0   1960     0 |    0.08   0.52  -0.08  -0.50
0  1.61e+05    2.21 |
0  1.61e+05    2.21 |    2.21    0.0   1960     0 |    0.03   0.61   0.03  

[W CudaIPCTypes.cpp:15] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]


Understanding the above results::
*   **Step**: the total training steps.
*  **MaxR**: the maximum reward.
*   **avgR**: the average of the rewards.
*   **stdR**: the standard deviation of the rewards.
*   **objA**: the objective function value of Actor Network (Policy Network).
*   **objC**: the objective function value (Q-value)  of Critic Network (Value Network).

# **Part 7: Backtest and Draw the Graph**

In [12]:
agent.if_on_policy = False
args = Arguments(env_eval, agent)

args.if_remove = False
args.cwd = './AgentPPO/StockTradingEnv-v1_0'
args.init_before_training()

env.draw_cumulative_return(args, torch)

| Keep cwd: ./AgentPPO/StockTradingEnv-v1_0


AttributeError: 'AgentPPO' object has no attribute 'ClassCri'